<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/Opt_T7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Definimos las funciones para calcular el gradiente y Hessiano de la función de Rosembrok.

In [2]:
def Rosembrok(X):#X es un vector.
    return np.sum(100*(X[1:]-X[:-1]**2)**2+(1-X[:-1])**2)
def Rosembok_grad(X):
    n=len(X)
    grad=np.zeros(n)
    grad[1:-1]=-400*X[1:-1]*(X[2:]-X[1:-1]**2)+200*(X[1:-1]-X[:-2]**2)-2*(1-X[1:-1])
    grad[0]=-400*X[0]*(X[1]-X[0]**2)-2*(1-X[0])
    grad[-1]=200*(X[-1]-X[-2]**2)
    return grad
def Rosembrok_Hessian(X):
    banda_s=X[:-1]
    H=np.diag(banda_s,1)
    H=-400*H+-400*H.T
    main_diag=-400*(X[2:]-X[1:-1]**2)+800*(X[1:-1]**2)+202
    np.fill_diagonal(H[1:-1,1:-1],main_diag)
    H[0,0]=(-400*(X[1]-X[0]**2)+800*(X[0]**2)+2)
    H[-1,-1]=200
    H[0,1]=-400*(X[1])
    H[-1,-2]=-400*X[-2]
    return(H)

Definimos las funciones para calcular el gradiente y Hessiano de a función de Wood.


In [3]:
def Wood(X):
    return (100*(X[0]**2-X[1])**2+(X[0]-1)**2+(X[2]-1)**2+90*(X[2]**2-X[3])**2+10.1*((X[1]-1)**2+(X[3]-1)**2)+19.8*(X[1]-1)*(X[3]-1))
def Wood_grad(X):
    G=[]
    G.append(400*X[0]*(X[0]**2-X[1])+2*(X[0]-1))
    G.append(-200*(X[0]**2-X[1])+20.2*(X[1]-1)+19.8*(X[3]-1))
    G.append(2*(X[2]-1)+360*(X[2]**2-X[3])*X[2])
    G.append(-180*(X[2]**2-X[3])+20.2*(X[3]-1)+19.8*(X[1]-1))
    return G
def Wood_Hessian(X):
    D1=[2+400*(X[0]**2-1)+800*X[0]**2,-400*X[0],0,0]
    D2=[-400*X[0],220.2,0,19.8]
    D3=[0,0,2+720*(X[2]**2)+360*(X[2]-X[3]),-360*X[2]]
    D4=[0,19.8,-360*X[2],200.2]
    return [D1,D2,D3,D4]

Definimos las funciones para calcular el gradiente y Hessiano de a función de Branin.


In [4]:
a=1
b=5.1/(4*np.pi**2)
c=5/np.pi
r=6
s=10
t=1/(8*np.pi)
def Branin(X):
    return (a*(X[1]-b*X[0]**2+c*X[0]-r)**2+s*(1-t)*np.cos(X[0])+s)
def Branin_grad(X):
    return [2*a*(X[1]-b*X[0]**2+c*X[0]-r)*(-2*X[0]+c)-s*(1-t)*np.sin(X[0]),2*a*(X[1]-b*X[0]**2+c*X[0]-r)]
def Branin_Hessian(X):
    R1=[2*a*(X[1]-b*X[0]**2+c*X[0]-r)*(-2*b)+(-2*b*X[0]+c)*(-4*a*b*X[0]+2*a*c)-s*(1-t)*np.cos(X[0]),2*a*(-2*b*X[0]+c)]
    R2=[2*a*(-2*b*X[0]+c),2*a]
    return [R1,R2]

A continuncación implementamos el método de región de confianza usando la modificación de Dog-leg.

In [ ]:
def conseguir_p_k():
    pass
def get_Newton(grad,B):
    return np.linalg.solve(B,-grad)
def get_Cauchy(grad,B):
    pass
def get_Newton_intersection(a,b,c):
    return ((-b+np.sqrt(b**2-4*a*c))/(2*a))
def get_Cauchy_wRestrictions(grad,B,radio):
    FC=np.dot(grad,np.dot(B,grad))
    if(FC<=0):
        tao=1
    else:
        tao=np.min(1,np.linalg.norm(grad)**3/(radio*FC))
    return tao

In [ ]:
def Dog_leg(x_0,f,grad_f,Hessian_f,radio_global,radio_0,etha_1=0.25,etha_2=0.75,tol_g=10**(-4),max_iter=10**3):
    #Conseguimos $p_k$
    
    ### Pendiente
    x_k=x_0
    radio_k=radio_global/2
    ####
    p_k_B=get_Newton(grad_f(x_k),Hessian_f(x_k))
    if (np.linalg.norm(p_k_B)>=radio_k):
        p_k_U=get_Cauchy(grad_f(x_k),Hessian_f(x_k))
        if(np.linalg.norm(p_k_U)>=radio_k):
            #Buscamos la interseccion
            pass#Creo es conocido P_k_C
            p_k=get_Cauchy_wRestrictions(grad_f(x_k),Hessian_f(x_k),radio_k)
        else:
            #Buscamos interseccion de Newton
            tao=get_Newton_intersection(a=np.linalg.norm(p_k_B-p_k_U)**2,b=2*np.dot(p_k_B,p_k_B-p_k_U),c=np.linalg.norm(p_k_U)**2-radio_k**2)
            p_k=p_k_U+tao*(p_k_B-p_k_U)
    else:
        p_k=p_k_B
    #Calcular medida de ajuste
    rho=(f(x_k)-f(x_k+p_k))/(m(0)-m(p_k))
    if(rho>etha_1):
        if(rho>etha_2 and np.norm(p_k)==radio_k):
        #Actualizamos el radio_k
            radio_k=np.min(2*radio_k,radio_global)
        #Actualizamos x_k
        x_k_next=x_k+p_k
        x_k=x_k_next
    else:
        radio_k=0.25*radio_k

